In [2]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm 
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data\events.csv')
atBats = []
for i in range(len(df)):
    if df['Away'][i].isdigit():
        atBats.append(1)
    else:
        atBats.append(0)
df["At Bat"] = atBats

In [4]:
pbp = df[df["At Bat"] == 1]

In [5]:
del pbp["At Bat"]


In [6]:
pbp.reset_index(drop=True, inplace=True)
inning = []
for i in range(len(pbp)):
    inning.append(re.findall(r'\d+', pbp['Inning'][i])[0])
pbp["Inning"] = inning

In [7]:
pbp.head()
pbp2= pbp

In [8]:
pbp = pbp2

In [9]:
events = ['out','hit by pitch','singled','doubled','tripled','advance','walked','stole','caught stealing','pickoff','picked off','homer','homerun','homerun','sacrifice fly','sacrificed','wild pitch','error','bunt','single','double','triple','struck out']
out = []
for event in events:
    pbp[event] = pbp['Events'].str.contains(event, na = False).astype(int)

In [10]:
pbpReserve = pbp

In [11]:
pbp.head()

,Game,Pitching Team,Batting Team,Inning,Event Id,Events,Away,Home,out,hit by pitch,...,homerun,sacrifice fly,sacrificed,wild pitch,error,bunt,single,double,triple,struck out
0,360403123,PIT,STL,1,1,Carpenter grounded out to first.,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,360403123,PIT,STL,1,2,Pham grounded out to third.,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,360403123,PIT,STL,1,3,Holliday struck out looking.,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,360403123,STL,PIT,1,5,Jaso grounded out to second.,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,360403123,STL,PIT,1,6,McCutchen hit by pitch.,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [12]:
pbp['homerun']=pbp['homer']+pbp['homerun']
pbp['pickoff']=pbp['caught stealing']+pbp['pickoff']
pbp['sacrifice']=pbp['sacrifice fly'] + pbp['sacrificed']
pbp['single'] = pbp['single']+pbp['singled']
pbp['double'] = pbp['double']+pbp['doubled']
pbp['triple'] = pbp['triple']+pbp['tripled']
pbp['advance'] = pbp['advance'] + pbp['wild pitch']+pbp['error']+pbp['bunt']
pbp['pickoff'] = pbp['pickoff'] + pbp['picked off']
pbp['out'] = pbp['out'] + pbp['sacrifice']+pbp['pickoff']



In [13]:
del pbp['homer']
del pbp['caught stealing']
del pbp['sacrifice fly']
del pbp['wild pitch']
del pbp['error']
del pbp['picked off']
del pbp['bunt']
del pbp['singled']
del pbp['doubled']
del pbp['tripled']

In [14]:
events = ['out','hit by pitch','single','double','triple','advance','walked','stole','pickoff','homerun','struck out']


In [15]:
pbp['sum'] = np.zeros(len(pbp))
for event in events:
    pbp['sum'] = pbp['sum'] + pbp[event]
pbp=pbp[pbp['sum']!=0]

In [16]:
pbp.head()

,Game,Pitching Team,Batting Team,Inning,Event Id,Events,Away,Home,out,hit by pitch,...,stole,pickoff,homerun,sacrificed,single,double,triple,struck out,sacrifice,sum
0,360403123,PIT,STL,1,1,Carpenter grounded out to first.,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.0
1,360403123,PIT,STL,1,2,Pham grounded out to third.,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.0
2,360403123,PIT,STL,1,3,Holliday struck out looking.,0,0,1,0,...,0,0,0,0,0,0,0,1,0,2.0
3,360403123,STL,PIT,1,5,Jaso grounded out to second.,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.0
4,360403123,STL,PIT,1,6,McCutchen hit by pitch.,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1.0


In [17]:
pbp['sum'] = pbp['out']-pbp['out']
for event in events:
    pbp['sum']=pbp['sum']+pbp[event]

In [18]:
pbp= pbp[pbp['sum']!=0]

In [19]:
del pbp['sum']

In [20]:
pbp.reset_index(drop=True, inplace=True)

In [21]:
pbp['Event Id'] = pbp.index 

In [22]:
endOfGame = np.zeros(len(pbp))
home_team = np.array(["asdf" for _ in range(len(pbp))])
away_team = np.array(["asdf" for _ in range(len(pbp))])
home_team_final_score = np.zeros(len(pbp))
away_team_final_score = np.zeros(len(pbp))

count = 0
games = pbp['Game'].unique()
for gameId in games:
    count = count + 1
    if count % 100 == 0:
        print(count/len(games))
    last_play = pbp[pbp['Game'] == gameId].iloc[-1]
    last_play_index= last_play['Event Id']
    endOfGame[last_play_index] = 1
    
    first_play = pbp[pbp['Game'] == gameId].iloc[0]
    away_team1=first_play['Batting Team']
    home_team1=first_play['Pitching Team']
    
    home_team[pbp['Game']==gameId]=home_team1
    away_team[pbp['Game']==gameId]=away_team1
    
    home_team_final_score[pbp['Game']==gameId]=last_play['Home']
    away_team_final_score[pbp['Game']==gameId]=last_play['Away']
    
    
    
pbp['End Of Game'] = endOfGame
pbp['Home Team'] = home_team
pbp['Away Team'] = away_team
pbp['Home Final'] = home_team_final_score
pbp['Away Final'] = away_team_final_score

0.007474400179385604
0.014948800358771209
0.02242320053815681
0.029897600717542418
0.03737200089692802
0.04484640107631362
0.05232080125569923
0.059795201435084835
0.06726960161447045
0.07474400179385604
0.08221840197324165
0.08969280215262725
0.09716720233201286
0.10464160251139847
0.11211600269078406
0.11959040287016967
0.12706480304955528
0.1345392032289409
0.14201360340832647
0.14948800358771208
0.1569624037670977
0.1644368039464833
0.1719112041258689
0.1793856043052545
0.1868600044846401
0.1943344046640257
0.20180880484341132
0.20928320502279693
0.2167576052021825
0.22423200538156812
0.23170640556095373
0.23918080574033934
0.24665520591972495
0.25412960609911056
0.26160400627849617
0.2690784064578818
0.27655280663726733
0.28402720681665294
0.29150160699603855
0.29897600717542416
0.3064504073548098
0.3139248075341954
0.321399207713581
0.3288736078929666
0.3363480080723522
0.3438224082517378
0.3512968084311234
0.358771208610509
0.3662456087898946
0.3737200089692802
0.381194409148665

In [23]:
pbp.head()

,Game,Pitching Team,Batting Team,Inning,Event Id,Events,Away,Home,out,hit by pitch,...,single,double,triple,struck out,sacrifice,End Of Game,Home Team,Away Team,Home Final,Away Final
0,360403123,PIT,STL,1,0,Carpenter grounded out to first.,0,0,1,0,...,0,0,0,0,0,0.0,PIT,STL,4.0,1.0
1,360403123,PIT,STL,1,1,Pham grounded out to third.,0,0,1,0,...,0,0,0,0,0,0.0,PIT,STL,4.0,1.0
2,360403123,PIT,STL,1,2,Holliday struck out looking.,0,0,1,0,...,0,0,0,1,0,0.0,PIT,STL,4.0,1.0
3,360403123,STL,PIT,1,3,Jaso grounded out to second.,0,0,1,0,...,0,0,0,0,0,0.0,PIT,STL,4.0,1.0
4,360403123,STL,PIT,1,4,McCutchen hit by pitch.,0,0,0,1,...,0,0,0,0,0,0.0,PIT,STL,4.0,1.0


In [24]:
pbp['Home Team Batting'] = (pbp['Home Team'] == pbp['Batting Team']).astype(int)

In [25]:
pbp['Batting Team Score'] = pbp['Home Team Batting']*pbp['Home'] +(1-pbp['Home Team Batting'])*pbp['Away']
pbp['Pitching Team Score'] = (1-pbp['Home Team Batting'])*pbp['Home'] +pbp['Home Team Batting']*pbp['Away']
pbp['Batting Team Final'] = pbp['Home Team Batting']*pbp['Home Final'] +(1-pbp['Home Team Batting'])*pbp['Away Final']
pbp['Pitching Team Final'] = (1-pbp['Home Team Batting'])*pbp['Home Final'] +pbp['Home Team Batting']*pbp['Away Final']

In [26]:
home_total_outs = np.zeros(len(pbp))
home_team_outs = np.zeros(len(pbp))
away_total_outs = np.zeros(len(pbp))
away_team_outs =np.zeros(len(pbp))
for gameId in games:
    away_team_outs[pbp['Game']==gameId] = pbp[pbp['Game']==gameId]['out']*(1-pbp[pbp['Game']==gameId]['Home Team Batting'])
    home_team_outs[pbp['Game']==gameId] = pbp[pbp['Game']==gameId]['out']*(pbp[pbp['Game']==gameId]['Home Team Batting'])
    away_total_outs[pbp['Game']==gameId] = away_team_outs[pbp['Game']==gameId].cumsum()
    home_total_outs[pbp['Game']==gameId] = home_team_outs[pbp['Game']==gameId].cumsum()
pbp["Home Outs"] = home_total_outs
pbp["Away Outs"] = away_total_outs

In [27]:
pbp["Batting Outs"] = pbp['Home Team Batting']*pbp['Home Outs'] +(1-pbp['Home Team Batting'])*pbp['Away Outs']
pbp["Pitching Outs"] = (1-pbp['Home Team Batting'])*pbp['Home Outs'] +(pbp['Home Team Batting'])*pbp['Away Outs']

In [28]:
pbp['Home Wins'] = (pbp["Home Final"] >pbp["Away Final"]).astype(int)
pbp['Batting Wins'] = (pbp["Batting Team Final"] >pbp["Pitching Team Final"]).astype(int)

In [29]:
Xfeatures = ['Batting Team Score','Pitching Team Score','Batting Outs','Pitching Outs']
Yfeatures = ['Batting Wins']

In [30]:
pbp = pbp.dropna()

In [31]:
X = np.array(pbp[Xfeatures])
Y = np.array(pbp[Yfeatures])

In [32]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression().fit(X, Y)

In [33]:
reg.score(X, Y)

0.7559085796760981

In [34]:
reg.predict_proba([[0,0,0,0],[0,0,1,0]])

array([[0.48765215, 0.51234785],
       [0.51697468, 0.48302532]])

In [35]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
Xtr, Xte, Ytr, Yte = train_test_split(X, Y, test_size=0.94, random_state=2718)

clf = MLPClassifier(random_state=2718, max_iter=300).fit(Xtr, Ytr)

In [36]:
clf.score(Xte, Yte)

0.7521536014505034

In [37]:
clf.predict_proba([[0,0,0,0],[0,0,1,0]])

array([[0.48751415, 0.51248585],
       [0.53838831, 0.46161169]])

In [38]:
np.sum(pbp['Home Wins'])/len(pbp)

0.5211574559231664

In [39]:
Xfeatures = ['Home','Away','Home Outs','Away Outs']
Yfeatures = ['Home Wins']

In [40]:
X = np.array(pbp[Xfeatures])
Y = np.array(pbp[Yfeatures])

In [41]:
reg = LogisticRegression().fit(X, Y)

In [42]:
reg.predict_proba([[0,0,0,0],[0,0,1,0]])

array([[0.45416252, 0.54583748],
       [0.47431145, 0.52568855]])

In [43]:
arr = {}
arr['Game'] = []
for event in events:  
    arr[event] = []
arr['Home Wins'] = []
for gameId in games:
    first_play = pbp[pbp['Game'] == gameId].iloc[0]
    arr['Game'].append(gameId)
    for event in events:
        arr[event].append(first_play[event])
    arr['Home Wins'].append(first_play['Home Wins'])
gamedf=pd.DataFrame(arr)
    


# Start reading here.

Essentially what I am doing is going through every MLB game since 2008 and calculating the odds that an event will occur.  Doing this with NBA data I found a pretty significant edge with a 2 way parlay.  I couldn't find any for MLB 2 way parlays so I am going to do 3 -> 4 ways.

This strategy should be used with a large amount of small bets over long periods of time. Also none of this is tested yet.  I am going to start testing soon which means writing down all my bets and seeing how much I make.  Also none of this is player specific so you should try balance your risk by betting on some opposite things happening.  For example your main bet could be First batter strikes out and away team wins but if that first batter is good at not striking out(I need to implement this part) then you should bet that that particular batter will strike out less than average amount of time.  I have yet to do to the math to show it but you should normalize the batter to being average in this way.  

This might be confusing.  Here is what you should do.  Bet on something that has good odds.  Bet a little against your own bet, so that if a team/player has different odds than average you can stay even.

###  Odds to probablity and probability to odds
These two functions convert odds to prob and prob to odds.  You can find the formulas online

In [44]:
def prob_to_odds(p):
    if p > .5:
        return -1*((-100*p)/(p-1)) 
    return (100-100*p)/p


def odds_to_prob(odds):
    if odds < 0:
        return abs(odds)/(abs(odds) + 100)
    return 100/(odds + 100)

### Parlay first at bat outcome + specific team winning
This calculates the odds of the first at bat and the home/away team winning.  

In [56]:

for event in events:
    print()
    print("Event to be parlayed with win: " + str(event))
    eventOccurs = gamedf[gamedf[event]==1]
    prob= np.sum(eventOccurs['Home Wins'])/len(eventOccurs)
    print(str(event) + ' and home team wins odds: ' + str(prob_to_odds(prob)))
    prob= np.sum(1-eventOccurs['Home Wins'])/len(eventOccurs)
    print(str(event) + ' and away team wins odds: ' + str(prob_to_odds(prob)))



Event to be parlayed with win: out
out and home team wins odds: -125.33267619517004
out and away team wins odds: 125.33267619517004

Event to be parlayed with win: hit by pitch
hit by pitch and home team wins odds: 133.33333333333334
hit by pitch and away team wins odds: -133.33333333333331

Event to be parlayed with win: single
single and home team wins odds: -120.40816326530611
single and away team wins odds: 120.40816326530611

Event to be parlayed with win: double
double and home team wins odds: 112.50000000000001
double and away team wins odds: -112.50000000000001

Event to be parlayed with win: triple
triple and home team wins odds: nan
triple and away team wins odds: nan

Event to be parlayed with win: advance
advance and home team wins odds: -125.26315789473685
advance and away team wins odds: 125.26315789473685

Event to be parlayed with win: walked
walked and home team wins odds: 101.31868131868131
walked and away team wins odds: -101.31868131868134

Event to be parlayed wit

### Observations
Obviosuly stole and Pickoff dont make sense so they are nan(meaning there is not enough data in this case 0).  
### Triples 
Triples just happen so little that I think there was too few data points and so it rounded to being impossible
### Advance
This is just any play description that says "advance" in it.  Not sure what that means but I would ignore it.

### Betting strategy
If you want to use this to bet.  Make sure you understand that is not player specific and bet accordingly.


# Ignore the code until the next Giant text

In [59]:
def getFeaturesByGame(df2,games,gameFeatures,firstPlayFeatures=[]):
    arr = {}
    if 'Game' not in gameFeatures:
        arr['Game'] = []
    else:
        gameFeatures.remove('Game')
        arr['Game'] = []
    for f in gameFeatures:
        arr[f] = []
    for f in firstPlayFeatures:
        arr[f] = []
    for gameId in games:
        arr['Game'].append(gameId)
        first_play = df2[df2['Game'] == gameId].iloc[0]
        
        for f in firstPlayFeatures:
            arr[f].append(first_play[f])
        for f in gameFeatures:
            arr[f].append(first_play[f])
    return pd.DataFrame(arr)


In [60]:
gameFeatures = ['Game','Batting Team Final','Pitching Team Final','Home Final','Away Final','Home Wins','Batting Wins']
scoreDf = getFeaturesByGame(pbp,games,gameFeatures)

In [61]:
loserScore = scoreDf['Home Wins']*scoreDf['Away Final'] + (1-scoreDf['Home Wins'])*scoreDf['Home Final']

In [62]:
(len(loserScore[loserScore >3.5])/len(loserScore))

0.32057702369384855

In [63]:
odds_to_prob(-150)

0.6

In [52]:
arr = {}
arr['Game'] = []
arr['Team Score'] = []
arr['Opp Score'] = []
arr['Won'] = []
for gameId in games:
    arr['Game'].append(gameId)
    arr['Game'].append(gameId)
    first_play = pbp[pbp['Game'] == gameId].iloc[0]
    arr['Team Score'].append(first_play['Home Final'])
    arr['Team Score'].append(first_play['Away Final'])
    arr['Opp Score'].append(first_play['Away Final'])
    arr['Opp Score'].append(first_play['Home Final'])
    arr['Won'].append(first_play['Home Wins'])
    arr['Won'].append(1-first_play['Home Wins'])


In [53]:
teamDf = pd.DataFrame(arr)

### Different betting batting strat
Here we were looking at betting that cards would win and that the the pirates would score > 3.5 runs

In [64]:
theirOddsB= -120
theirOddsTotal = 370
myOdds = (len(teamDf[teamDf['Opp Score']>3.5][teamDf['Won']==1])/len(teamDf[teamDf['Opp Score']>3.5]))*odds_to_prob(theirOddsB)
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))


My Odds 554.8962462112381
Their Odds 370


### The implied odds say this is awful.  I think betting against this would probably be beneficial if I could

### No edge when going with lower score. But it gets better

In [56]:
theirOddsB= -525
theirOddsTotal = 130
myOdds = (len(teamDf[teamDf['Opp Score']>1.5][teamDf['Won']==1])/len(teamDf[teamDf['Opp Score']>1.5]))*odds_to_prob(theirOddsB)
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))


My Odds 188.08392959947363
Their Odds 130


### Better but not good enough

In [57]:
theirOddsB= -2500
theirOddsTotal = -120
myOdds = (len(teamDf[teamDf['Opp Score']>.5][teamDf['Won']==1])/len(teamDf[teamDf['Opp Score']>.5]))*odds_to_prob(theirOddsB)
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))


My Odds 121.77804114111571
Their Odds -120


### Now lets look at unders runs 

In [102]:
totalRuns = 6.5
theirOddsB= 190
theirOddsTotal = -160
teamDf['Total Score'] = teamDf['Team Score']+teamDf['Opp Score']
conditionScore = teamDf[teamDf['Team Score'] < totalRuns][teamDf['Opp Score'] < totalRuns]
print(len(teamDf))
print(len(conditionScore))
myOdds = (len(conditionScore[conditionScore['Won']==1])/len(conditionScore))*odds_to_prob(theirOddsB)
myOdds2 = (len(conditionScore[conditionScore['Won']==1])/len(conditionScore))*(len(conditionScore)/len(teamDf))
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))
print(prob_to_odds(myOdds2))


26758
1520
My Odds 480.0
Their Odds -160
3420.7894736842104


In [95]:
odds_to_prob(480.0)

0.1724137931034483

In [97]:
odds_to_prob(-160)

0.6153846153846154

In [76]:
prob_to_odds(.999)

-99899.99999999991

In [ ]:
mets_over = 8.5
phillies_under = 5.5
theirOddsB= -170
theirOddsC= 450
theirOddsTotal = 750

subevents_occuring = odds_to_prob(theirOddsB) * odds_to_prob(theirOddsC)
#teamDf['Total Score'] = teamDf['Team Score']+teamDf['Opp Score']
conditionScore = teamDf[teamDf['Team Score'] < totalRuns][teamDf['Opp Score'] < totalRuns]
myOdds = (len(conditionScore[conditionScore['Won']==1])/len(conditionScore))*subevents_occuring
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))
print(prob_to_odds(myOdds2))


In [74]:
arr = {}
arr['Game'] = []
arr['Team Score'] = []
arr['Opp Score'] = []
arr['Won'] = []
for gameId in games:
    arr['Game'].append(gameId)
    #arr['Game'].append(gameId)
    first_play = pbp[pbp['Game'] == gameId].iloc[0]
    arr['Team Score'].append(first_play['Home Final'])
    #arr['Team Score'].append(first_play['Away Final'])
    arr['Opp Score'].append(first_play['Away Final'])
    #arr['Opp Score'].append(first_play['Home Final'])
    arr['Won'].append(first_play['Home Wins'])
    #arr['Won'].append(1-first_play['Home Wins'])
stdf = pd.DataFrame(arr)

# START READING HERE AGAIN
So we are going to betting on this mets game today.  
First we are going to check on some lines that I suspect people don't really think about.  If a team scores a lot it actually affect how much the other team scores.

The block below works in the following way:
conditionscore is the amount of games where the home team scores more than 8.5 points.
We then get the number of games in which the opponent scores under 5.5 points.  



The way this works is bayes rule which you can look up if you want.  
Our goal is to find the probablity that both events will occur.

So we find the probability that the second event occurs given that the first event already occurred.  Then you simply multiply that probability by the probability tthat the first event occurred.

In [81]:
mets_over = 8.5
phillies_under = 5.5
theirOddsB= -280
theirOddsC= 450
theirOddsTotal = 750

subevents_occuring = odds_to_prob(theirOddsC) 

conditionScore = stdf[stdf['Team Score'] > mets_over]
myOdds = (len(conditionScore[stdf['Opp Score'] < phillies_under])/(len(conditionScore))*subevents_occuring)
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))



My Odds 738.8999008919722
Their Odds 750


### Success
We found a good bet.  Now Just do this a bunch of time

In [92]:
mets_over = 8.5
phillies_under = 6.5
theirOddsB= -280
theirOddsC= 450
theirOddsTotal = 650

subevents_occuring = odds_to_prob(theirOddsC) 

conditionScore = stdf[stdf['Team Score'] > mets_over]
myOdds = (len(conditionScore[stdf['Opp Score'] < phillies_under])/(len(conditionScore))*subevents_occuring)
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))



My Odds 666.1688311688312
Their Odds 650


While I was doing this they changed the Odds lmao.  Now Mets > 8.5 has odds 475.  So now this bet is bad so dont make it

In [106]:
over = 8.5
under = 5.5
theirOddsB= -170
theirOddsC= 475
theirOddsTotal = 600

subevents_occuring = odds_to_prob(theirOddsC) 

conditionScore = stdf[stdf['Team Score'] > over]
myOdds = (len(conditionScore[stdf['Opp Score'] < under])/(len(conditionScore))*subevents_occuring)
print('My Odds ' + str(prob_to_odds(myOdds)))
print('Their Odds ' + str(theirOddsTotal))



My Odds 777.0317145688801
Their Odds 600


## You can see how this is works but you essentially have to search for these types of bets.  B